## Park 1. Checking data availability

1. Install AWS CLI:
```
$ curl "https://awscli.amazonaws.com/AWSCLIV2.pkg" -o "AWSCLIV2.pkg" \n
$ sudo installer -pkg AWSCLIV2.pkg -target /
```
2. Get the SRR IDs by following the instructions [here](https://github.com/asadprodhan/A-Guide-to-Automatically-Downloading-NCBI-SRA-Reads?tab=readme-ov-file#step-2-collect-sra-accession-numbers). (Also stored under `data/srr_acc_list.txt`)
3. Install python requirements: `pip3 install -r requirements.txt`
4. Check availability of SRR data: `python3 scripts/check_aws_availability.py`

## **Part 2: Setting up Amazon EC2 instance**

## **Launch EC2 Instance from Console**

### **Step 1: Go to EC2**
- Sign in to AWS Console
- Search for "EC2" in the top search bar
- Click "EC2"

### **Step 2: Launch Instance**
- Click the orange "Launch instance" button

### **Step 3: Configure Instance**

**Name:**
- Enter: `mgs-download`

**Application and OS Images (AMI):**
- Select "Ubuntu Server 22.04 LTS"
- Make sure it says "Free tier eligible"

**Instance type:**
- Select `t3.large` from the dropdown

**Key pair:**
- Click "Create new key pair"
- Name: `mgs-key`
- Key pair type: RSA
- Private key file format: `.pem`
- Click "Create key pair"
- **Save the downloaded file somewhere safe**

**Network settings:**
- Leave defaults (should auto-create security group with SSH access)

**Configure storage:**
- Change from `8` GB to `500` GB
- Leave type as "gp3"

### **Step 4: Launch**
- Click "Launch instance" (orange button at bottom right)
- Wait 2-3 minutes for instance to start

### **Step 5: Set Reminder to Terminate (IMPORTANT)**
**Before continuing, set an reminder to terminate EC2 instance, otherwise you will get surprised by a high bill**

### **Step 6: Get Connection Info**
- Click "View all instances"
- Find your `mgs-download` instance
- Wait until "Instance state" shows "Running"
- Click on the instance ID
- Copy the "Public IPv4 address" (looks like `3.123.45.67`)

### **Step 7: Connect from Your Terminal**

```bash
# Navigate to where you saved mgs-key.pem
cd ~/Downloads  # or wherever you saved it

# Make key secure
chmod 400 mgs-key.pem

# SSH in (replace with your actual IP)
ssh -i mgs-key.pem ubuntu@3.123.45.67
```

## **Part 2: Install SRA Toolkit on EC2**

### **Step 1: Update Package Manager**

```bash
sudo apt update
```

### **Step 2: Install SRA Toolkit**

```bash
sudo apt install sra-toolkit -y
```

### **Step 3: Verify Installation**

```bash
fastq-dump --version
```

Should show version number (e.g., `fastq-dump : 2.11.0`).

### **Step 4: Configure SRA Toolkit**

```bash
vdb-config --interactive
```

This opens a text-based GUI. Use arrow keys and Enter to navigate:

1. Press `TAB` to move between sections
2. Navigate to "CACHE" tab
3. Enable "local file-caching" (if not already enabled)
4. Set cache location to `/home/ubuntu/ncbi/public` (or leave default)
5. Navigate to "AWS" tab  
6. **Disable** "report cloud instance identity" (uncheck it)
7. Press `S` to Save
8. Press `X` to Exit


## **Part 3:Add Git Repo to EC2 Instance**

### **Step 1: Install Git**

```bash
sudo apt install git -y
```

### **Step 2: Clone Your Repository**

```bash
git clone https://github.com/farhanazam98/metagenomic-sequencing-exercise.git
```

### **Step 3: Navigate to Project**

```bash
cd metagenomic-sequencing-exercise
```

## **Part 4:Download SRA Files**

### **Background: What These Commands Do**

**fasterq-dump:**
- Tool that downloads and converts SRA files to FASTQ format
- Multi-threaded (faster than old `fastq-dump`)
- Gets data from NCBI's servers

**--split-files:**
- Separates paired-end reads into two files
- Creates `_1.fastq` (forward reads) and `_2.fastq` (reverse reads)
- Required for paired-end data

**FASTQ format:**
- Standard text format for storing sequencing reads
- Contains: sequence + quality scores
- One read = 4 lines (ID, sequence, +, quality)

**gzip:**
- Compresses files to save space (~5× smaller)
- `.fastq.gz` files can still be read by analysis tools
- Essential when working with large datasets

---

### **Step 1: Make Script Executable**

```bash
chmod +x scripts/download_sra.sh
```

### **Step 2: Run the Download**

```bash
./scripts/download_sra.sh
```

**Time estimate:** 1-2 hours for all 18 samples

### **Step 3: Verify Downloads**

```bash
ls -lh data/raw/fastq/
```

You should see 36 files (18 samples × 2 files each), all ending in `.fastq.gz`

---

**Downloads will be saved to** `data/raw/fastq/` directory.

## **Part 5: Install Python Dependencies and Run Check Script**

### **Step 1: Install pip (Python package manager)**

```bash
sudo apt install python3-pip -y
```

### **Step 2: Install Required Python Packages**

```bash
pip3 install pandas tqdm
```

**What these do:**
- `pandas` - reads CSV files and data manipulation
- `tqdm` - progress bars for loops

### **Step 3: Run the Check Script**

```bash
python3 scripts/check_downloads.py
```

**Expected output:**
- Lists each SRR as "Found" or "Not found"
- Shows summary: Downloaded / Missing / Total

---

**If all show "Not found"** - that's expected if you haven't downloaded anything yet.